In [ ]:
#@title Data-AI（必ず自分の名前・学籍番号を入力すること） { run: "auto", display-mode: "form" }

import urllib.request as ur
import urllib.parse as up
Name = '\u6C5F\u6D32\u51FA\u4E95 \u592A\u90CE' #@param {type:"string"}
EName = 'Esudei Taro' #@param {type:"string"}
StudentID = '87654321' #@param {type:"string"}
Addrp = !cat /sys/class/net/eth0/address
Addr = Addrp[0]
url = 'https://class.west.sd.keio.ac.jp/classroll.php'
params = {'class':'dataai','name':Name,'ename':EName,'id':StudentID,'addr':Addr,
           'page':'dataai-text-3','token':'71873536'}
data = up.urlencode(params).encode('utf-8')
#headers = {'itmes','application/x-www-form-urlencoded'}
req = ur.Request(url, data=data)
res = ur.urlopen(req)

---
> デジタルデータなんて所詮0か1さ。とはいえ、0と1の羅列だけでは意味をなさない。その意味やルールを与えたのは人間、自分が作った意味やルールに振り回されるなんて、滑稽じゃあないか。
---

# データの扱い

機械学習以外でも、データを整理したり、まとめたり、内容を知るといった処理は重要である\
ここでは、データを扱う・確認する・補正するという観点についてまとめる

## まずは準備

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

# データ取得


## データセットの読み込み

ここでは、scikit-learnに含まれているデータセットを利用する

ここで利用するのは、次のデータである

- Linnerud (生理学的特徴と運動能力の関係)  
ノースカロライナ州立大学の A. C. linnerud 博士が作成した、20 人の成人男性に対してフィットネスクラブで測定した 3 つの生理学的特徴と 3 つの運動能力の関係を表すデータである

> データセットの詳細

| | |
|:-:|:-:|
|レコード数|	20|
|カラム数|	説明変数:3, 目的変数: 3|
|主な用途|	多変数回帰 (multivariate regression)|

> 説明変数の構成

| | |
|:-:|:-:|
|Weight	体重
|Waist	ウエスト (胴囲)
|Pulse	脈拍

> 目的変数の構成

| | |
|:-:|:-:|
|Chins|	懸垂の回数|
|Situps|	腹筋の回数|
|Jumps|	跳躍|

データの利用には、`from パッケージ名 import ライブラリ名`として、ライブラリが複数まとめられたパッケージから特定のライブラリをインポートする

なお、
- sklearn.datasets：scikit-learnに組み込まれているデータセットのパッケージ
- load_linnerud：linnerudデータセットを用いるためのライブラリ
である

In [ ]:
from sklearn.datasets import load_linnerud
linnerud = load_linnerud()

ここで、説明変数としてWeight(体重)、Waist(胴囲)、Pulse(脈拍)の三つ、目的変数としてChins(懸垂の回数)、Situps(腹筋の回数)、Jumps(跳躍)を与える

linnerud.dataは以下のようなarray型のデータが表示される
```
array([[   5.,  162.,   60.],  
       [   2.,  110.,   60.],  
       ...
       [  12.,  101.,  101.],  
```

linnerud.targetを実行すると以下のようなarray型のデータが表示される
```
array([[ 191.,   36.,   50.],
       [ 189.,   37.,   52.],
       [ 193.,   38.,   58.],
       ...
```

In [ ]:
linnerud.target_names

In [ ]:
linnerud.data

In [ ]:
linnerud.target

データの表示方法には複数存在することに注意する
- array：Pythonで使われる配列である。なお、型は無表示である
- ndarray：配列同様、多次元のデータを格納できる構造でnumpyと対応している。なお、型はarrayと表示される
- DataFrame：2次元のデータを格納できる構造．pandasと対応している．なお、型はpandas.core.frame.DataFrameと表示される

相互変換であるが、

- pythonリスト型listをNumPy配列ndarrayに変換するにはnumpy.array()とする
- NumPy配列ndarrayをリスト型listに変換には、tolist()メソッドを使う
- ndarrayから、pandasの配列に変換するには、DataFrameメソッドを利用する



pd.DataFrame(numpyデータなど, columns=カラム名)  
- 一番左の1列をインデックスと呼び、自動でナンバリングされる。ただし0オリジンであることに注意すること
- 一番上の1行をカラムと呼び、それぞれをカラム名を定義する必要がある
  
linnerud.feature_namesを実行すると以下のようなarray型のデータが表示される

`['Chins', 'Situps', 'Jumps']``
- Chins：懸垂の回数
- Situps：腹筋の回数
- Jumps：跳躍

In [ ]:
data = pd.DataFrame(linnerud.data, columns=linnerud.feature_names)
data.head(5)

linnerud.target_namesを実行すると以下のようなarray型のデータが表示される 
```['Weight', 'Waist', 'Pulse']```
- Weight：体重
- Waist：胴囲
- Pulse：脈拍
である

In [ ]:
target = pd.DataFrame(linnerud.target, columns=linnerud.target_names)
target.head(5)

## DataFrameの作成

dataとtargetを横方向に結合しdatargetという名前のDataFrameを作成する



- A.join(B)
- pd.concat([A, B], axis=1)
- pd.merge(A, B, right_index=True, left_index=True)
    - AもBもDataFrame

今回はjoinがもっとも素直な方法である。なお、how='outer'は省略できる

In [ ]:
datarget1 = data.join(target, how='outer')
datarget1.head(3)

concatは、indexが共通かつ、axis=1を省略すると縦方向の連結になる

inner joinなど様々な結合方法があるので整理しておくと良い

In [ ]:
datarget2 = pd.concat([data, target], axis=1)
datarget2.head(3)

In [ ]:
datarget2d = pd.concat([data[0:2], target[0:2]], axis=0)
datarget2d

本来、margeは共通のキーがあるときに利用する。したがって、今回は無理やりな例になる

In [ ]:
datarget3 = pd.merge(data, target, right_index=True, left_index=True)
datarget3.head(3)

 ## CSVデータの読み込み
 
CSVからデータを読み込む場合について補足する

- ここでは、numpyが提供するloadtxtについて説明する

例えば、1列目が目的変数、2列目以降が説明変数となる

```
foo.csv
1,1.2,12,...
1,2.0,5,...
1,1.5,3,...
2,1.8,1,...
2,0.5,10,...
2,1.0,8,...
```

というCSVファイルがあった場合、

```
data = np.loadtxt('foo.csv', delimiter=',', dtype=float)
labels = data[:, 0:1] # 目的変数を取り出す
features = preprocessing.minmax_scale(data[:, 1:]) # 説明変数を取り出した上でスケーリング
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3) # トレーニングデータとテストデータに分割
```

とするとよい




### numpyでのコロン : およびカンマ , による配列表記

最初のコロンはスライシングのコロン(`x[1:]`におけるコロンと同じ)である。Pythonではスライシングはカンマで区切り複数記載できる

次の例では、2から5まで2飛ばし、つまり、2と4個目の要素がスライシングで選ばれる

- つまり、0から数えるので`[[7,  8,  9], [13, 14, 15]]`となる。この0列目なので、`[7,13]`となる

- 要するに、第一行だけがほしい場合は`a[0]`、第一列だけがほしい場合は`a[:,0]`となる

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]])
a[2:5:2, 0]

データの読み込み、確認手法について以上である

# データ加工

## 欠損値補完

実際に用いるデータでは欠損値が含まれることが普通にある
- データを扱うには欠損値を処理しておく必要がある
- 上記データは欠損値がないため、欠損値のあるデータを用いて説明する

今回は、自動車事故のデータセットを使う

ここでQuilt(キルト)を利用する。Quiltはデータセットの公開およびバージョン管理のためのツールであり、Python,Pandas,Jupyter等を使って機械学習の開発・研究をするユーザのために設計されている

In [ ]:
has_quilt = !if [[ -f /usr/local/bin/quilt ]]; then echo 1; fi;
if not has_quilt:
  !pip install -q quilt
  !quilt install --force ResidentMario/missingno_data

Quiltにある自動車事故データセットをメモリ内に取り込む。

In [ ]:
from quilt.data.ResidentMario import missingno_data
collisions = missingno_data.nyc_collision_factors()

中身を確認する。

In [ ]:
collisions.head(2)

大量の欠損値があるが、欠損値がnanとして記されているので、これを、データとしてのnan(np.nan)へ変更する

In [ ]:
collisions.replace("nan", np.nan, inplace=True)

表示させると、nanが、NaNに変わったことがわかる

- NaNは、numpyにおけるNaNという特別な値を意味し、欠損値であることを示している
- nanは文字であり、「本当にnanという意味のある情報、例えばカレーはriceかnanかというアンケート」と混乱する場合がある
  - そこで、あえて特別な値を用いる必要がある

- NaN（Not a Number、非数、ナン）は、コンピュータにおいて、主に浮動小数点演算の結果として、不正なオペランドを与えられたために生じた場合を表す値またはシンボルであり、計算機における数字の表現の一つとして標準化されている

In [ ]:
collisions.head(2)

In [ ]:
collisions.shape

### 欠損値の数を把握する

データの数が示されているが、総数は7303で、それに満たないデータには欠損があることがわかる
- かなりのデータが欠損していることがわかる

In [ ]:
collisions.info()

### 欠損の状況を把握する

方針をたてよう

- まとめてデータが欠落していれば(バースト欠損)、そこだけ除くと良いであろう

- ほぼランダムに欠損がある場合は、当該場所の値を補完すればよいであろう
  - 補完には、重回帰解析による推定値で保管するなどが考えられる

- いずれにしても、欠損の場所や欠落状況を把握することは重要

pythonでは、様々な可視化ツールが用意されており、これらを利用できるメリットは大きい

- 一般的なのはseaborn.heatmapを用いて視覚的に解析することであるが、ここではさらに強力なツールを紹介する

こういう便利なツールが揃っており作る必要がない点もpythonやscikit-learnを利用する重要なメリット
- 存在を知っていること、もっといえば、更新が速いため、そういった更新や新ツールへの適応が重要である

様々な可視化ができるmissingnoというmoduleがある
- Google Colaboratoryではデフォルトでインストールされている
- こちらの方が見やすく、また、正確で機能が高い
- 使えるものをわざわざ作るな！作る時間があれば使いこなせ！ないなら作れ！作る状況がでるぐらいに学べ！

In [ ]:
import missingno as msno
msno.matrix(collisions)

右側のバーは、各行ごとに非欠損値の個数を線グラフとして表示しており、欠損値の重なりが多いとグラフは小さな値になる\
**非欠損値であることに注意**すること

さらに、heamapメソッドを利用することで欠損値と発生箇所の相関もわかる。見方の概要は次のとおりである

- ヒートマップには相関値が示される。-1から1の値をとり、丸めて0になる場合 (>-0.05 or < 0.05)、数値は表示されない

- 欠損値のない値はヒートマップに現れない

- -1の場合は左のカラムが欠損している一方で、下のカラムが全て欠損していないことを示す

- 1の場合は、左のカラムが欠損しており、さらに、下のカラムも全て欠損していることを表す

In [ ]:
msno.heatmap(collisions)

より地道な方法として、単純にisnull()で NULLの数を調べ、それをsum()で数え上げてみる

In [ ]:
collisions.isnull().sum()

それぞれのカラム毎にどのくらいのnullが存在し、また値のバリエーションがどの程度あるか調べるには次のようにすると良い
- `count()`: 欠損値NaNではない要素の数をpandas.Seriesやスカラー値として取得する
- `isnull()`: isnaと同じで欠損値であればTrueになる
- `sum()`： Trueつまり1の数を数え上げるため、Trueの総数を計算することになる

In [ ]:
def chknull(df):
  for i in df.columns:
    nall = df[i].count()
    nnul = df[i].isnull().sum()
    if(nall == 0):
      np = "-"
    else:
      np = str(round(float(nnul)/float(len(df))*100, 2))
    print("* " + i + "\t#:" + str(nall) +
      "\t# of NULL:" + str(nnul)+
      "\t% of NULL:" + np +
      "\t# of Orig:" + str(df[i].value_counts().count()))
chknull(collisions)

## 削除

削除には、pandasのdropnaメソッドを利用する

#### まとめて削除する
- `how='all'`とするとすべての値が欠損値である行が削除される
- `axis=1`とすると、すべての値が値が欠損値である列が削除される
- `how='any'`とすると、一つでも欠損値がある行が削除され、行ではaxis=1が利用できる
- `thresh=3`とすると欠損値ではない要素の数が3個以上含まれている行が残り、それ以外の行（欠損値ではない要素の数が2個以下の行）が削除される

なお、行も列も消したい場合の推奨方法は、メソッドを2回呼ぶことである

```
df.dropna(how='all').dropna(how='all', axis=1)
```

#### 狙って削除する

特定の行・列を基準に削除したい場合は、引数subsetに対象とする行ラベル・列ラベルをリストで指定する\
例えば`subset=['name']`と指定する

デフォルトではsubsetで指定した列のいずれかに欠損値がある行を削除する

- `how='all'`とすると、指定した列すべてが欠損値である行のみを削除する
- `axis=1`とすると、subsetで指定した行に欠損値がある列を削除することができ、引数howも利用できる

ZIP CODEがない場合、BOROUGH（自治区）もないため、単純に削除する\
削除した結果をcollisions_r1 に代入する

In [ ]:
collisions_r1 = collisions.dropna(subset=['ZIP CODE'])

全く値の入っていない列があるため、これらを削除し、削除した結果をcollisions_r2に入れる
- NUMBER OF CYCLISTS INJURED	などが削除されていることを確認する

In [ ]:
collisions_r2 = collisions_r1.dropna(how='all', axis=1)
collisions_r2.head(2)

In [ ]:
collisions_r2.shape

## 補完

データ量が少ない場合などでは、単純に削除するとデータ量が少なくなりムダかつ妥当性や学習結果に支障が出る可能性がある
- そこで、なにかしらデータを補完する
- データを眺めたあと、どのような形で補完するかを決定すること

以下に単純な数値データの補完方法について述べる

- ラベル情報（文字情報など）の補完には、クラスタリングといった手法が別途必要となり、特に大量に補完する場合は注意が必要である

そもそも、その準備したデータに問題がないのかを常に疑うこと
- 大量に補完したデータを学習に利用すれば、「学習で補完したデータを使って学習する」つまり、学習に都合の良いデータを使って学習しており、本来の意味が薄れている
- ここでは、kaggleを使う

これで、
- Scikit-learn 付属のデータセット
- Quiltのデータセット
- Kaggleのデータセット

以上が利用できるようになったはずである

特にKaggleは、「The Home of Data Science & Machine Learning」（データサイエンスと機械学習の家）と題されている通り、世界中の機械学習・データサイエンスに携わっている約40万人の方が集まるコミニティーである

- 企業や政府などの組織とデータ分析のプロであるデータサイエンティスト/機械学習エンジニアを繋げるプラットフォームであり、企業や政府がコンペ形式（競争形式）で課題を提示し、賞金と引き換えに最も精度の高い分析モデルを買い取るといったサービスが提供されている
- Kaggleコンペで上位に入ると、著名企業からハンティングされる可能性あり

その中でもよく学習や例題に使われる、Titanic: Machine Learning from Disasterを利用する
- これは、著名な豪華客船タイタニック号が沈没したときの乗客員のデータである
- 多くの乗客が死亡してしたため、後から確認できない項目も多く欠損値が含まれるデータとなっている

[https://www.kaggle.com/c/titanic](https://www.kaggle.com/c/titanic)本来はこちらから入手するが、入手にはアカウントの登録が必要である

- アクセスしたら、Dataタブを選び、Data Sourcesから、test.csvとtrain.csvをクリックして、ダウンロードする

同じデータは、こちらにも配置しているので、以下を用いるとよい
- なお、コードは自動でダウンロードするように設計しているのでダウンロードする必要はない
  - [train.csvはここをクリックするとダウンロードできる](http://class.west.sd.keio.ac.jp/dataai/data/train.csv)
  - [test.csvはここをクリックするとダウンロードできる](http://class.west.sd.keio.ac.jp/dataai/data/train.csv)

簡単に済ますには、次のセルを実行して取得し、/contentの中に保存する
- データを読み込むと自動的に/contentの中に入る
- 読み込んだら、ファイルから、train.csvをクリックして中身を見て、Ageの欠損について確認する

In [ ]:
import os
if not os.path.exists('train.csv'):
  #!wget "https://drive.google.com/uc?export=download&id=1OnwqCkFYr49GuEeB6NSHt7kzOOEy80oF" -O train.csv
  !wget https://keio.box.com/shared/static/h4xfiaehi5vt9exmz246gnvm23qnpjlb -O train.csv
if not os.path.exists('test.csv'):
  #!wget "https://drive.google.com/uc?export=download&id=1OosaY5iW9O00IR-TwJQMb3o1ydVUG33a" -O test.csv
  !wget https://keio.box.com/shared/static/plxazmlkvooo35id3hcetdixm39r2gs6 -O test.csv

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

### 単純に平均値/最頻値を入れる
単純に平均値/最頻値をいれてしまう方法である
- まずデータをコピーしている
- pythonで=を使って値をコピーすると痛い目に合うのは**あるある**である
  - `a = b`はコピーではなく参照、つまり同じ変数の保存場所をどちらも指しているので、bを変更するとaも変更されてしまう
  - `a = b.copy()`とするとaは別メモリに保存されbの変更の影響を受けない
  - ところが、オブジェクトの中にオブジェクトがある場合(配列の配列など)は、残念ながらその中のオブジェクトを変更すると変更されてしまう
  - そこで、`import copy`として、`a = copy.deepcopy(b)`とすると、完全に中身も含めてコピーされる

In [ ]:
import copy
train_df_original = copy.deepcopy(train_df)
plt.hist(train_df_original["Age"].dropna(), alpha=0.2,color="r") #もともとのグラフを赤で描画
plt.hist(train_df["Age"].fillna(train_df["Age"].mean()),alpha=0.2,color="b") #平均値をいれたグラフを青で描画

### 平均からばらつきを考慮して補完する
平均値から標準偏差でばらつきを考慮して補完する方法を示す
- まずは、共通する処理として、平均・標準偏差・null数を取得しておく

In [ ]:
Age_average = train_df["Age"].mean() #平均値
Age_std = train_df["Age"].std()  #標準偏差
Age_nullcount = train_df["Age"].isnull().sum() #null値の数＝補完する数

次に、正規分布に従うとし、標準偏差の範囲内でランダムに数字を作る

In [ ]:
rand = np.random.randint(Age_average - Age_std, Age_average + Age_std , size = Age_nullcount)
#Ageの欠損値
train_df_rand = copy.deepcopy(train_df_original)
train_df_rand["Age"][np.isnan(train_df["Age"])] = rand
#グラフ描画
plt.hist(train_df_original["Age"].dropna(), alpha=0.2,color="r")
plt.hist(train_df_rand["Age"],alpha=0.2,color="b")

### 補完の考え方

上記の補完方法を採用してよいかどうかは、それぞれの問題に即して考えなければならない

欠損値の生成過程に関して、次の3パターンのいずれであるかを考慮する必要がある

- MCAR(Missing completely at random)

 完全にランダムに欠損しているパターンであり、例えば、データシートにコーヒーをこぼしたや、メモリに宇宙線がランダムに打ち込まれ、データが化けたといった場合である
 - この場合の対処は容易であり、上記の方法で問題ない

- MAR(Missing at random)

 そのデータの他の特徴量に依存して欠損するパターンであり、例えば、日本人の信仰は"無"が多いなど、そもそも欠損することが別の理由で一般的な場合を指す
  - そもそも、欠損していることが普通であることから、補完せず、欠損そのものに別のラベルを与えるべきであろう

- MNAR(Missing not at random)

 欠損となった値自体に依存して欠損するパターンであり、例えば、かなり古いがさだまさしの歌にあるように、O型だから(馬鹿にされることが多く)血液型を答えたくない場合もあるであろう
 - この場合の対処は難しく、欠損値がどの生成パターン由来かは、可視化して確かめる必要があり、適切に対処されなければならない

### 対策のまとめ

- 削除手法
 - リストワイズ：欠損値を含むデータを削除
 - ペアワイズ：2つの特徴量の相関をプロットする際などに、NaNがあり不都合が生じる計算だけ無視する。
- 単一代入補完手法
 - 統計量の代入：平均値や最頻値、中央値などを代入し補完
 - 回帰代入法：欠損値の無いサンプルから回帰して補完
 - 確率的回帰代入法：回帰代入法の結果にノイズを加えて補完
- 完全情報最尤推定手法
 - FIML(full maximum likelihood method)：最尤推定(with EM)で補完
- 多重代入補完  MI(multiple imputation)
 - 欠損値を様々な単一代入補完したデータセットを複数作成し、各データセットで分析を行い、その結果を統合し欠損値を補完
- 機械学習的なアプローチ
 - weighted k-nearest neighbour algorithm (kNN)：kNNでいくつか近傍データを探してきて、重み付け和で欠損値を補完
 - Random Forestを用いた欠測データの補完とその応用による方法：RのMissForestや、FIML・MIといった手法
 

## 実践的な補完方法

### interpolate()の基本的な使い方
以下のpandas.DataFrameを例に補完方法について述べる

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'col1': [0, np.nan, np.nan, 3, 4],
                   'col2': [np.nan, 1, 2, np.nan, np.nan],
                   'col3': [4, np.nan, np.nan, 7, 10]})
df

デフォルトでは各列に対して線形補間を行う
- 下端の欠損値には同じ値が繰り返される
- 上端の欠損値はそのままとなる

In [ ]:
df.interpolate()

引数axis=1とすると各行に対して補間される
- 右端の欠損値には同じ値が繰り返される
- 左端の欠損値はそのままとなる


In [ ]:
df.interpolate(axis=1)

引数limitにより、欠損値が連続している場合、最大でいくつの欠損値を補間するかを指定することができる
- デフォルトはNoneで連続する欠損値すべてが補間される

In [ ]:
df.interpolate(limit=1)

補間方向は引数limit_directionで'forward', 'backward', 'both'のいずれかを指定する
- デフォルトは'forward'である

In [ ]:
df.interpolate(limit=1, limit_direction='forward') # col2,0はNaNのまま

In [ ]:
df.interpolate(limit=1, limit_direction='backward') # col2,0が補完された

In [ ]:
df.interpolate(limit=1, limit_direction='both')


補間対象領域は引数limit_areaで指定する。'inside'だと内挿のみ、'outside'だと外挿のみ、None（デフォルト）だと両方が対象となる
- 外挿については上述のlimit_directionで前方（上側・左側）、後方（下側・右側）、両方を指定できる

In [ ]:
df.interpolate(limit_area='inside')

In [ ]:
df.interpolate(limit_area='outside')

In [ ]:
df.interpolate(limit_area='outside', limit_direction='both')

では、完全に補完つまり、NaNを無くすにはどうすればよいか？

In [ ]:
df.interpolate(limit_direction='both')

- **おわかりいただけただろうか**

補間方法は第一引数methodに指定する
- デフォルトはmethod='linear'で線形補間である
  - まず、次のデータを準備する

In [ ]:
s = pd.Series([0, np.nan, np.nan, np.nan, 4, np.nan, np.nan],
              index=[0, 2, 5, 6, 8, 10, 14])
s

In [ ]:
s.interpolate()

method='linear'（デフォルト）ではインデックス列が数値でも特に考慮されないが、method='index'またはmethod='values'とするとインデックス列を考慮して補間される
- 但し、indexが数字であることが必須である

In [ ]:
s.interpolate('index')

In [ ]:
s.interpolate('values')

method='spline'とするとスプライン補間を行う
- 同時に引数orderに次数を指定する必要がある

これまでのsの値ではスプライン補完ができないので、新たに次の値を用いて確認する
- スプライン補間は常にインデックス列を考慮して補間される

In [ ]:
s = pd.Series([0, 10, np.nan, np.nan, 4, np.nan, np.nan],
              index=[0, 2, 5, 6, 8, 10, 14])
s.interpolate('spline', order=2)

### fillnaの例

まず、fillnaは、nanを埋めるための専用の関数である

直前の値を使って埋めていく


In [ ]:
df.fillna(method='ffill')

直後の値を使って穴埋めをする

In [ ]:
df.fillna(method='bfill')

平均値(df.mean())を用いて補完する方法は既に述べたが、他に中央値(df.median())や、最頻値(df.mode())を用いて補完する方法がある
- medianについては次の通りである

In [ ]:
df.fillna(df.median())

### その他

補間方法としては、そのほか、'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'krogh', 'polynomial', 'piecewise_polynomial', 'from_derivatives', 'pchip’, ‘akima'が指定可能
- 特にtimeについて説明する
- 次のようなデータがあった場合、タイムスタンプに応じて補完される

In [ ]:
df_nan = pd.DataFrame({'value': [1, np.nan, np.nan, np.nan, 31]},
                      index=pd.to_datetime(['2018-01-01', '2018-01-02', '2018-01-15', '2018-01-20', '2018-01-31']))
print(df_nan)
print(df_nan.interpolate())
print(df_nan.interpolate('time'))

# 課題3(データ補完の効果)

**[課題1]** 次のコードをコピーして実行し、結果も含めたノートブックを作成して提出しなさい

**[課題2]** pandasのシンプルな補完を使って動作を確認してみなさい

上記を一つのノートブックにして提出しなさい

**注意点**

ノートブックには次の内容を先頭に必ず記述すること

- 先頭セルはテキストで、

 「# **データシステムの知能化とデザイン**」と記載
- さらに「# 第3回課題」と記載
- 次に、「## 学籍番号」と「## 氏名」を記載

これは、全ての課題に共通する事項であり、忘れずに記載すること

まずはデータの準備をする
- またしても、作業ゲーである

pandasの補完もかなり強力であるが、さらに高度な補完を行うため、scikit-learnのIterativeImputerを用いる
- fancyimputeという補完ライブラリも存在するので確認するとよい

今回はメルボルンの住宅データを利用する
- どういうデータが入っているかだけ確認する

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
#使用カラムを限定する
usecols=['Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Distance', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']
if not os.path.exists('Melbourne_housing_FULL.csv'):
    !wget "https://drive.google.com/uc?export=download&id=1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt" -O Melbourne_housing_FULL.csv

df=pd.read_csv("Melbourne_housing_FULL.csv",usecols=usecols)


内容を確認する
- Pandas形式ならば安心してprintなしで全部表示してよい
- きちんと省略してくれる

In [ ]:
df

このデータについて、家賃以外の情報から家賃を当ててみよう
- なお、データには、そもそもPriceが設定されていないデータが存在していることがわかるので、これを削除しなければならない

In [ ]:
df.dropna(subset=['Price'])

マニアックだが、次のようにしても同じ結果が得られる
- これは、`==`の比較演算子が、np.nanの比較に対してFalseを返すということを知っているからできる方法

In [ ]:
df[df["Price"]==df["Price"]]

In [ ]:
df.dropna(subset=['Price'], inplace=True) #inplaceでdfそのものを修正する
print("全データ数")
print(len(df["Rooms"])) # Roomsに欠損がないことを知っているので
print("\n欠損数の確認")
print(df.isnull().sum())
#カテゴリ値を数値変換
le = preprocessing.LabelEncoder()
df[[ 'Type', 'Method', 'SellerG']]=df[[ 'Type', 'Method', 'SellerG']].apply(le.fit_transform)
df_train=df.drop("Price",axis=1)
df_label=df["Price"].values
print("\n訓練データの確認")
print(df_train)
print("\n訓練ラベルの確認(つまりお部屋のお値段、これを当てる)")
print(df_label)

マイクロソフトのLightGBMを利用して実際に学習させる
- 今回は、補完による性能への影響を見たいので、全て関数にしておく
- スコア(ロス関数の値)と、主要特徴量の一覧を返すが、ここでは主要特徴量は特に利用していない
  - 「どの特徴量が価格に大きな影響を与えるか」を影響力と共に知ることができるので、各自で試みると良い

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import copy
def run_model(df_train,df_label):
    params={
        'objective': 'regression',
        'random_state' : 1,
        "metric": "rmse",
        }
    kfold = 5
    score=0
    kf = KFold(n_splits=kfold,shuffle=True,random_state=94) #交差検証する関数を定義、データをkfold個にわけて1つをテスト用、残りを訓練に使う
    importance=0
    for i, (train_index, test_index) in enumerate(kf.split(df_train, df_label)): #定義したKFoldでdf_trainとdf_labelを分割
        print('[Fold %d/%d]' % (i + 1, kfold))
        X_train, X_valid = df_train.iloc[train_index], df_train.iloc[test_index] #train_indexなどは「選択した要素の配列」0から要素数あり、kfoldの確率で抜けている
        y_train, y_valid = df_label[train_index], df_label[test_index] #上と同様だが、要素が一つしかないのでこれでよい
        dtrain = lgb.Dataset(X_train, label=y_train) #まとめてデータセットを構成する
        dvalid = lgb.Dataset(X_valid, label=y_valid)
        bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid],early_stopping_rounds=50,verbose_eval=100)
        # LigntBGMで学習、1000回試行して、ベストなものを返す
        importance += pd.DataFrame(bst.feature_importance(), index=df_train.columns, columns=['importance'])
        score+=bst.best_score["valid_1"]['rmse']
    return importance,score/5

では、まず最初に補完せずに、そのまま欠損を含むデータを利用して学習させてみる
- 意味合いとしては、欠損を欠損というデータとして扱うことになる
- 欠損に意味があれば、これもわるくないが、無作為に欠損する場合は場合は補完した方がよくなると考えられる

In [ ]:
imp,score = run_model(df_train,df_label)
print(score)
scored = score

In [ ]:
#欠損値を補完関数IterativeImputerのデフォルトのBaysian Ridgeで補完
imp = IterativeImputer(max_iter=50, random_state=5) # 方法の定義
df_train_fi = pd.DataFrame(imp.fit_transform(df_train)) # 定義した方法を用いて実際に補完
df_train_fi.columns = df_train.columns # カラム情報もコピーしておく
imp,score = run_model(df_train_fi,df_label)
print(score)
scorei = score

In [ ]:
#欠損値を補完関数IterativeImputerのKNeighborsRegressorで補完
from sklearn.neighbors import KNeighborsRegressor
imp = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=15),max_iter=10,random_state=0)
df_train_fk = pd.DataFrame(imp.fit_transform(df_train))
df_train_fk.columns = df_train.columns
imp,score = run_model(df_train_fk,df_label)
print(score)
scorek = score

結果をみてみよう
- 結果は、RMSEであり、小さい方が良い結果といえる

In [ ]:
print("結果")
print("何もしない", scored)
print("Baysian Ridge", scorei)
print("KNeighborsRegressor", scorek)

補完に時間のかかる高度な方法を採用したからと言って、結果が良くなるとは限らない
- もちろん、これがLightGBMの特徴といえるかもしれない
- 欠損補完の目的の一つは、「欠損していることを特徴として利用するのではなく、統計的観点からバイアスなく統計処理を行う」ことにあるため、目的を見極めて補完の利用を考える必要がある

では、Pandasで補完してみよう

In [ ]:
#pandasで補完
dfp = df_train.interpolate(limit_direction='both')
df_train_fp=pd.DataFrame(dfp)
df_train_fp.columns=df_train.columns
imp,score=run_model(df_train_fp,df_label)
print(score)
scorep = score

In [ ]:
print("結果")
print("何もしない", scored)
print("Baysian Ridge", scorei)
print("KNeighborsRegressor", scorek)
print("Pandasで補完", scorep)

さすがにpandasのinterpolateはだめだが、なんとなにもしないデータが最高性能をたたき出した
- そういうこともあるよという例
- おおよそこの授業は、まっとうな結果ではなく、どちらかというとイレギュラーな結果を示していることに注意すること
  - 今後様々なイレギュラーに遭遇すると思うが、それに対応できるように

**(意欲的な人向け)**

作業ゲーは嫌だという人向けに、以下、あくまでも任意ということで

- 何が原因かをデータの特徴やアルゴリズムの観点から調査してみよう
- このことから、どのような「背景」つまり、「住戸価格」という現実的な話において、何がいえるであろうか
  - そのことは普遍的に(例えば他の街や国でも)いえるだろうか
  - 結局、現実に何が起きているのかを把握することが重要であろう